### Intial setup

Installation of libraries

In [1]:
!pip install -U autogen-agentchat autogen-ext[openai]
!pip install torch transformers sentencepiece

  Using cached jsonref-1.1.0-py3-none-any.whl.metadata (2.7 kB)
     ---------------------------------------- 0.0/66.6 kB ? eta -:--:--
     ---------------------------------------- 66.6/66.6 kB 1.8 MB/s eta 0:00:00
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/105.5 kB ? eta -:--:--
   ---------------------------------------- 105.5/105.5 kB 3.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/94.7 kB ? eta -:--:--
   ---------------------------------------- 94.7/94.7 kB 5.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/662.0 kB ? eta -:--:--
   ------------------- -------------------- 317.4/662.0 kB 6.5 MB/s eta 0:00:01
   ---------------------------------------- 662.0/662.0 kB 6.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/893.9 kB ? eta -:--:--
   ---------------------------- --

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.3 requires pydantic<2.0, but you have pydantic 2.11.4 which is incompatible.
googleapis-common-protos 1.63.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.29.4 which is incompatible.
google-api-core 2.18.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.29.4 which is incompatible.
langchain 0.3.3 requires langchain-core<0.4.0,>=0.3.10, but you have langchain-core 0.1.43 which is incompatible.
langchain-community 0.3.2 requires langchain-core<0.4.0,>=0.3.10, but you have langchain-core 0.1.43 which is incompatible.
langchain-community 0.3.2 requires langsmith<0.2.0,>=0.1.125, but you have langsmith 0.1.48 which is incompatible.

   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   - ------------------------------------- 41.0/991.5 kB 991.0 kB/s eta 0:00:01
   --------------------- ------------------ 532.5/991.5 kB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 991.5/991.5 kB 9.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


API key setup

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

### Multi Agent Setup

In [3]:
from autogen.agentchat import AssistantAgent

c:\Users\viraj\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


#### Fine tuned Agents

wrapper agent for autogen

In [23]:
from autogen.agentchat import AssistantAgent

class LocalModelWrapperAgent(AssistantAgent):
    def __init__(self, name: str, model_agent):
        """
        name: The AutoGen agent name (e.g., "semantic-agent", "contrast-agent").
        model_agent: Any class that implements `handle(raw_json: str) -> str`.
        """
        super().__init__(name=name)
        self.model_agent = model_agent

    def generate_reply(self, messages):
        # Expecting the last user message to contain the JSON string
        raw_json = messages[-1]["content"]
        return self.model_agent.handle(raw_json)


Semantic Agent

In [ ]:
from agents.semantic_agent.agent import SemanticAgent

semantic_model = SemanticAgent(model_dir="trusha88/t5-semantic-agent")
semantic_agent = LocalModelWrapperAgent(name="semantic-agent", model_agent=semantic_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Contrast Agent

In [27]:
from agents.contrast_agent.agent import ContrastAgent

contrast_model = ContrastAgent(model_dir="virajns2/contrast-violation-t5")
contrast_agent = LocalModelWrapperAgent(name="contrast-agent", model_agent=contrast_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### GPT based Agents

In [10]:
# Visually Impaired Agent
visually_impaired_agent = AssistantAgent(
    name="VisuallyImpairedAgent",
    system_message="Simulate a user navigating via screen reader. Identify missing labels, improper heading structures, and ARIA attribute issues.",
    llm_config={"model": "gpt-4", "temperature": 0}
)

In [11]:
# Motor-Impaired Agent
motor_impaired_agent = AssistantAgent(
    name="MotorImpairedAgent",
    system_message="Simulate keyboard-only navigation. Detect elements not accessible via keyboard and lack of focus indicators.",
    llm_config={"model": "gpt-4", "temperature": 0}
)


In [12]:
# Color-Blind Agent
color_blind_agent = AssistantAgent(
    name="ColorBlindAgent",
    system_message="Analyze color usage for accessibility. Identify issues with color contrast and reliance on color cues.",
    llm_config={"model": "gpt-4", "temperature": 0}
)

In [13]:
fixing_agent = AssistantAgent(
    name="FixingAgent",
    system_message="Provide code-level fixes for identified accessibility issues based on best practices and guidelines, based on the problems stated by everyone",
    llm_config={"model": "gpt-4", "temperature": 0}
)

Configure Agent Interactions

In [28]:
from autogen.agentchat import UserProxyAgent, GroupChat, GroupChatManager

user_proxy = UserProxyAgent(
    name="UserProxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    code_execution_config={"use_docker": False}
)

group_chat = GroupChat(
    agents=[user_proxy, semantic_agent, contrast_agent, visually_impaired_agent, motor_impaired_agent, color_blind_agent, fixing_agent],
    messages=[],
    max_round=5
)

manager = GroupChatManager(groupchat=group_chat, llm_config={"model": "gpt-4"})

Run the Evaluation

In [35]:
#TODO: FETCH RESPECTIVE DATA FROM TEST SET AND PASS TO SYSTEM
with open("test_data/test.json", "r", encoding="utf-8") as f:
    ui = f.read()
ui = "role: link, fg: 120,120,120, bg: 255,255,255, contrast: 2.9"



Feed back loop

In [36]:
user_proxy.initiate_chat(manager, message = ui)

for msg in group_chat.messages:
    print(f"{msg['role']}: {msg['content']}\n")
    if msg['role'] != 'FixingAgent':
        fixing_agent.receive_message(msg)

UserProxy (to chat_manager):

role: link, fg: 120,120,120, bg: 255,255,255, contrast: 2.9

--------------------------------------------------------------------------------

Next speaker: ColorBlindAgent

ColorBlindAgent (to chat_manager):

The color usage in this case has a contrast ratio of 2.9. According to the Web Content Accessibility Guidelines (WCAG), the minimum contrast ratio for normal text should be 4.5:1. Therefore, the contrast ratio here is too low, which could make it difficult for people with visual impairments, including color blindness, to read the text. 

Additionally, if there are any important cues or information conveyed solely through color, this would be inaccessible to those who are color blind or have other visual impairments. It's important to ensure that information is available through other means, such as text or patterns, in addition to color.

--------------------------------------------------------------------------------

Next speaker: FixingAgent

Fixi

TypeError: LocalModelWrapperAgent.generate_reply() got an unexpected keyword argument 'sender'